In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import plotly.graph_objects as go
#import matplotlib.pyplot as plt
#import seaborn as sns
#%matplotlib inline

In [2]:
summoner = 'Saikki kusuo'
key = 'RGAPI-9724b32a-f354-408c-8cde-8fdcc35e01fa'

## Extracting Functions

In [3]:
# Summoner v4, /lol/summoner/v4/summoners/by-name/{summonerName}
# Get summoner info by name using API key and summoner name

def get_summoner_info(summoner_name, apikey):
    # This function takes the summoner name and the API Key
    # and retrieves its account id
    url = f'https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summoner_name}'
    html = requests.get(url,
                       params={'api_key': apikey})
    json = html.json()
    df = pd.DataFrame(json, index=[0])
    
    return df

summoner_info = get_summoner_info(summoner, key)
accountid = summoner_info['accountId'][0]
accountid

'djQlAoXhpBz5y5YjoYJkpS-kVvZQx4Caj8DElKzSEPmJo8c'

In [4]:
def get_matchlist(accountid, api):
    
    url = f'https://euw1.api.riotgames.com/lol/match/v4/matchlists/by-account/{accountid}'
    html = requests.get(url,
                       params={'api_key': key})
    json = html.json()
    df = pd.DataFrame(json['matches'])
    
    return df

#match_list = get_matchlist(accountid, key)
#match_list

In [5]:
def get_match_info(matchid):
    
    url = f'https://euw1.api.riotgames.com/lol/match/v4/matches/{matchid}'
    html = requests.get(url,
                    params={'api_key': key})
    json = html.json()
    
    return json

match_info = get_match_info(4834424471)
#match_info.keys()

In [6]:
def get_players_info(json):
        
    players = pd.DataFrame()
    
    for participant in range(10):
        df = pd.DataFrame.from_dict(json['participants'][participant]['stats'], orient='index').T
        df.insert(loc=0, column='championId', value= json['participants'][participant]['championId'])
        df.insert(loc=1, column='teamId', value=json['participants'][participant]['teamId'])
        
        dff = pd.DataFrame.from_dict(json['participantIdentities'][participant]['player'], orient='index').T
        dff.insert(loc=0, column='participantId', value= json['participantIdentities'][participant]['participantId'])
        df_final = pd.merge(df, dff, on='participantId')
        
        players = players.append(df_final)
    
    players.insert(loc=0, column='gameid', value=[json['gameId'] for value in range(len(players))])
        
        
        
    return players

#players_info = get_players_info(match_info)
#players_info

In [7]:
def get_match_timeline(matchid):

    url= f'https://euw1.api.riotgames.com/lol/match/v4/timelines/by-match/{matchid}'
    html = requests.get(url,
                       params={'api_key': key})
    json = html.json()
    
    return json

timeline = get_match_timeline(4834424471)

In [8]:
def participant_frames(json):
    ## Extracting frames from json

    all_frames = pd.DataFrame()

    for frame in list(range(len(json['frames']))):
        iterable = json['frames'][frame]['participantFrames']
        for elem in iterable:
            dic = {}
            for sub_elem in iterable[elem]:
                if type(iterable[elem][sub_elem]) == dict:
                    dic.update(iterable[elem][sub_elem])
                else:
                    dic[sub_elem] = iterable[elem][sub_elem]
            df = pd.DataFrame(dic, index=[0])
            df.insert(loc=0, column='timestamp', value=json['frames'][frame]['timestamp'])
            all_frames = all_frames.append(df, ignore_index=True)

    cols = ['timestamp', 'participantId', 'x', 'y', 'currentGold', 'totalGold', 'level',
            'xp', 'minionsKilled', 'jungleMinionsKilled',
            # 'dominionScore','teamScore',
            ]
    all_frames = all_frames[cols]

    return all_frames

#frames = participant_frames(timeline)
#frames

In [9]:
def get_events(json):
    
    events = pd.DataFrame()
    
    frames = json['frames']
    for elem in range(len(frames)):
        events = events.append(pd.DataFrame(frames[elem]['events']))
    
    
    events['position_x'] = [elem.get('x') if type(elem) == dict else 'none' for elem in events['position']]
    events['position_y'] = [elem.get('y') if type(elem) == dict else 'none' for elem in events['position']]
    events.drop(columns='position', inplace=True)
    
    return events
    
#events = get_events(timeline)
#display(list(events))
#display(events)

In [10]:
summoner_info = get_summoner_info(summoner, key)
summoner_info

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel
0,iqJz7QEm4AJIHuEKecIyvBZ65qaANfGcYM9iYL-S_FT2ddQ,djQlAoXhpBz5y5YjoYJkpS-kVvZQx4Caj8DElKzSEPmJo8c,9XoYu4fgD-ybcBawfY2XsMsGgvv9Av44llBjAhHFZk-nX5...,Saikki Kusuo,4249,1602858817000,193


In [11]:
match_list = get_matchlist(accountid, key)
match_list

,platformId,gameId,champion,queue,season,timestamp,role,lane
0,EUW1,4877596885,202,420,13,1603120271250,DUO_CARRY,BOTTOM
1,EUW1,4877250033,98,440,13,1603102542421,SOLO,TOP
2,EUW1,4876228611,236,440,13,1603038875713,SOLO,MID
3,EUW1,4876149459,875,440,13,1603036266971,DUO_SUPPORT,BOTTOM
4,EUW1,4876010396,98,440,13,1603033356757,SOLO,TOP
...,...,...,...,...,...,...,...,...
95,EUW1,4836181863,51,450,13,1601108697813,DUO_SUPPORT,MID
96,EUW1,4836141451,79,420,13,1601108087423,DUO,NONE
97,EUW1,4835000843,79,420,13,1601046011714,NONE,JUNGLE
98,EUW1,4834424471,84,420,13,1601023541845,SOLO,MID


In [12]:
list(match_list['gameId'].head(10))

[4877596885,
 4877250033,
 4876228611,
 4876149459,
 4876010396,
 4875263982,
 4875169709,
 4874220844,
 4872347007,
 4872058149]

In [13]:
players_info = get_players_info(match_info)
players_info

,gameid,championId,teamId,participantId,win,item0,item1,item2,item3,item4,...,statPerk1,statPerk2,platformId,accountId,summonerName,summonerId,currentPlatformId,currentAccountId,matchHistoryUri,profileIcon
0,4834424471,150,100,1,False,1055,3071,3022,3075,3111,...,5008,5002,EUW1,cxpp71Wk6A8e5g-Oe_w2SLKCbFbJe-cqXt9TRkiyoNS7gg,gavilan93,5naoSf9Tuxgd8LpC7FgvvhOHdEjJi5S6mTjsmAgPerDVN_M,EUW1,cxpp71Wk6A8e5g-Oe_w2SLKCbFbJe-cqXt9TRkiyoNS7gg,/v1/stats/player_history/EUW1/27312474,3233
0,4834424471,875,100,2,False,3748,1401,3111,3076,3742,...,5008,5002,EUW1,yo29J_PdeL-809dylLKGpMb-o4YyAGhgbE8mhJdzwvyqv30,xSlein,0j4-M3pyKdStielmJHmJN9_PlSC7itPMhqaQZeCTATHFkcM,EUW1,yo29J_PdeL-809dylLKGpMb-o4YyAGhgbE8mhJdzwvyqv30,/v1/stats/player_history/EUW1/212478720,3807
0,4834424471,117,100,3,False,3853,3504,3117,3174,3114,...,5008,5002,EUW1,HpPKtIGVN34YdUekXjP5GQ0l1fSsTfk7w8FSALSFW1z4zTk,SeifReturn,IK9QQ6IuibJjFJjfwdqLgq7a7MQs-IqXPWUEGyIHpKPZyXjo,EUW1,HpPKtIGVN34YdUekXjP5GQ0l1fSsTfk7w8FSALSFW1z4zTk,/v1/stats/player_history/EUW1/239688242,4656
0,4834424471,67,100,4,False,1055,3153,2421,3124,3046,...,5008,5002,EUW1,zmMT0SToH4oLvXPSuFwvizfD-7WC9lwD86AsAdv8dmG0cw,Akimari,xlZAfVeGPL-lNOw3LppkckT9SAnH8gqio9w36QK5oBISW5k,EUW1,zmMT0SToH4oLvXPSuFwvizfD-7WC9lwD86AsAdv8dmG0cw,/v1/stats/player_history/EUW1/37244478,4656
0,4834424471,45,100,5,False,3020,3030,3157,1058,3285,...,5008,5002,EUW1,Ok-hbQOVEt9x2SVYnKZSbycWeM5fZc-7JAyKLu7gRG33Ww,personel,-a6BUFuLyO3vRV6pkUKkIwJ3MlGuZWvAWqlilReKlDx1ncs,EUW1,Ok-hbQOVEt9x2SVYnKZSbycWeM5fZc-7JAyKLu7gRG33Ww,/v1/stats/player_history/EUW1/33347436,582
0,4834424471,44,200,6,True,3110,3190,3860,3117,3050,...,5002,5002,EUW1,Jyt5n2Rwtsm8LHYD2_Ae5JTvshfiTwwBRXYi3mrdH_bo9Us,Skíron,eVRPsYgXgBFQDod5uCwftXj9HwUnIey4AT6dyRExrfSTt7OP,EUW1,Jyt5n2Rwtsm8LHYD2_Ae5JTvshfiTwwBRXYi3mrdH_bo9Us,/v1/stats/player_history/EUW1/219285021,2076
0,4834424471,106,200,7,True,1054,3068,3111,3076,3078,...,5008,5002,EUW1,mxBWMdjd0VhMSJ0lBBhaW6rplFs4waLEjHHNa-JECsdmAw,Sir Cuncision,BO30gVocWNUa8Ho4LcRBRCa3SuNyEhgaaOW-g3P9q0iAnaA,EUW1,mxBWMdjd0VhMSJ0lBBhaW6rplFs4waLEjHHNa-JECsdmAw,/v1/stats/player_history/EUW1/41198240,3151
0,4834424471,202,200,8,True,3026,3095,3009,3031,3094,...,5008,5002,EUW1,D1X8dw_Mcg11-4Uo53wEyStLvCQGl0xbd3JA3h6tE2ep9A,6ixnine,XfV6AAkCgoUZ9Og0qH-g7IAW0mr4NsydtBmbGGYSs_bWpx0,EUW1,D1X8dw_Mcg11-4Uo53wEyStLvCQGl0xbd3JA3h6tE2ep9A,/v1/stats/player_history/EUW1/24947195,4057
0,4834424471,84,200,9,True,1056,3157,3146,3151,3020,...,5008,5003,EUW1,djQlAoXhpBz5y5YjoYJkpS-kVvZQx4Caj8DElKzSEPmJo8c,Saikki Kusuo,iqJz7QEm4AJIHuEKecIyvBZ65qaANfGcYM9iYL-S_FT2ddQ,EUW1,djQlAoXhpBz5y5YjoYJkpS-kVvZQx4Caj8DElKzSEPmJo8c,/v1/stats/player_history/EUW1/229546365,4249
0,4834424471,141,200,10,True,1400,3812,3047,3071,3076,...,5008,5002,EUW1,Pw9vzObhdOJVNkgu5F3d_P74bT4wflfFOa2kJU3cS-NajF0,Hacker404,H2cx9tNUCLYghIaQtX4LZsmaGq9OFL2S3ogBWuugx11JGd0,EUW1,Pw9vzObhdOJVNkgu5F3d_P74bT4wflfFOa2kJU3cS-NajF0,/v1/stats/player_history/EUW1/202459306,4512


In [14]:
list(players_info)

['gameid',
 'championId',
 'teamId',
 'participantId',
 'win',
 'item0',
 'item1',
 'item2',
 'item3',
 'item4',
 'item5',
 'item6',
 'kills',
 'deaths',
 'assists',
 'largestKillingSpree',
 'largestMultiKill',
 'killingSprees',
 'longestTimeSpentLiving',
 'doubleKills',
 'tripleKills',
 'quadraKills',
 'pentaKills',
 'unrealKills',
 'totalDamageDealt',
 'magicDamageDealt',
 'physicalDamageDealt',
 'trueDamageDealt',
 'largestCriticalStrike',
 'totalDamageDealtToChampions',
 'magicDamageDealtToChampions',
 'physicalDamageDealtToChampions',
 'trueDamageDealtToChampions',
 'totalHeal',
 'totalUnitsHealed',
 'damageSelfMitigated',
 'damageDealtToObjectives',
 'damageDealtToTurrets',
 'visionScore',
 'timeCCingOthers',
 'totalDamageTaken',
 'magicalDamageTaken',
 'physicalDamageTaken',
 'trueDamageTaken',
 'goldEarned',
 'goldSpent',
 'turretKills',
 'inhibitorKills',
 'totalMinionsKilled',
 'neutralMinionsKilled',
 'neutralMinionsKilledTeamJungle',
 'neutralMinionsKilledEnemyJungle',
 'to

In [15]:
frames = participant_frames(timeline)
frames['timestamp'] = (frames['timestamp'] / 60000).astype('int')
frames

,timestamp,participantId,x,y,currentGold,totalGold,level,xp,minionsKilled,jungleMinionsKilled
0,0,1,560.0,581.0,500,500,1,0,0,0
1,0,2,560.0,361.0,500,500,1,0,0,0
2,0,5,351.0,293.0,500,500,1,0,0,0
3,0,4,221.0,471.0,500,500,1,0,0,0
4,0,3,311.0,649.0,500,500,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
355,35,7,NaN,NaN,1148,11356,16,16005,162,0
356,35,10,NaN,NaN,303,11928,15,13986,33,133
357,35,9,NaN,NaN,3409,15634,17,18178,218,15
358,35,8,NaN,NaN,3172,19672,17,17591,165,40


In [16]:
frames['timestamp'].max()

35

In [18]:
teams = players_info[['participantId','teamId']]
frames = pd.merge(frames, teams, on='participantId')

In [19]:
frames

,timestamp,participantId,x,y,currentGold,totalGold,level,xp,minionsKilled,jungleMinionsKilled,teamId
0,0,1,560.0,581.0,500,500,1,0,0,0,100
1,1,1,5425.0,9124.0,0,500,1,0,0,0,100
2,2,1,2037.0,12042.0,63,563,1,181,2,0,100
3,3,1,2638.0,12051.0,353,853,3,752,12,0,100
4,4,1,394.0,461.0,905,1405,4,1226,16,0,100
...,...,...,...,...,...,...,...,...,...,...,...
355,31,6,10284.0,5264.0,1243,10368,14,11449,39,0,200
356,32,6,4800.0,6134.0,1877,11002,15,13068,39,0,200
357,33,6,7395.0,10045.0,118,11443,15,13868,39,0,200
358,34,6,4541.0,1278.0,547,11872,15,14483,40,0,200


In [17]:
events = get_events(timeline)
events

,type,timestamp,participantId,itemId,skillSlot,levelUpType,wardType,creatorId,killerId,victimId,...,afterId,beforeId,monsterType,monsterSubType,teamId,buildingType,laneType,towerType,position_x,position_y
0,ITEM_PURCHASED,4033,1.0,1055.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,none,none
1,ITEM_PURCHASED,4429,1.0,2003.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,none,none
2,ITEM_PURCHASED,4858,1.0,3340.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,none,none
3,ITEM_PURCHASED,5122,10.0,1039.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,none,none
4,ITEM_PURCHASED,5684,10.0,2031.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,none,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,WARD_PLACED,2068188,NaN,NaN,NaN,NaN,UNDEFINED,8.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,none,none
12,ITEM_PURCHASED,2068519,10.0,3076.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,none,none
13,ITEM_DESTROYED,2068519,10.0,1029.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,none,none
14,ITEM_DESTROYED,2068519,10.0,1029.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,none,none


In [29]:
data = frames.groupby(['timestamp', 'teamId']).sum().reset_index()[['timestamp', 'teamId', 'totalGold']]
data

,timestamp,teamId,x,y,currentGold,totalGold,level,xp,minionsKilled,jungleMinionsKilled
0,0,100,2003.0,2355.0,2500,2500,5,0,0,0
1,0,200,71633.0,72005.0,2500,2500,5,0,0,0
2,1,100,37878.0,31304.0,0,2500,5,0,0,0
3,1,200,48518.0,46693.0,0,2500,5,0,0,0
4,2,100,36472.0,28908.0,386,2886,6,968,6,6
...,...,...,...,...,...,...,...,...,...,...
67,33,200,43289.0,41148.0,4801,65981,79,75195,574,183
68,34,100,22106.0,5788.0,3820,60618,75,69795,522,148
69,34,200,19079.0,11521.0,7477,68657,80,78636,605,188
70,35,100,0.0,0.0,3222,61032,75,69878,522,148


In [41]:
team100gold = data[data['teamId'] == 100]
team100gold.rename(columns={"totalGold": "team100gold"}, inplace=True)
team200gold = data[data['teamId'] == 200][['timestamp', 'teamId', 'totalGold']]
team200gold.rename(columns={"totalGold": "team200gold"}, inplace=True)

/home/david/miniconda3/envs/ironhack/lib/python3.7/site-packages/pandas/core/frame.py:4301: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [42]:
golddiff = pd.merge(team100gold, team200gold, on='timestamp')
golddiff['golddiff'] = golddiff['team100gold'] - golddiff['team200gold']
golddiff

,timestamp,teamId_x,x,y,currentGold,team100gold,level,xp,minionsKilled,jungleMinionsKilled,teamId_y,team200gold,golddiff
0,0,100,2003.0,2355.0,2500,2500,5,0,0,0,200,2500,0
1,1,100,37878.0,31304.0,0,2500,5,0,0,0,200,2500,0
2,2,100,36472.0,28908.0,386,2886,6,968,6,6,200,3058,-172
3,3,100,36528.0,33728.0,1813,4313,13,3284,34,16,200,4391,-78
4,4,100,24192.0,9112.0,2141,5801,16,5156,50,16,200,6649,-848
5,5,100,43618.0,31082.0,3052,7612,20,7129,66,20,200,7755,-143
6,6,100,38631.0,24815.0,3523,8983,23,9188,90,28,200,8991,-8
7,7,100,39602.0,26509.0,4214,10539,26,10959,110,28,200,10262,277
8,8,100,34876.0,30777.0,4207,11832,27,12614,129,36,200,12497,-665
9,9,100,34038.0,33754.0,5620,13245,32,15135,152,52,200,14226,-981


In [27]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data['timestamp'],
                         y=data['totalGold'])
)